In [1]:
import requests
import re
from bs4 import BeautifulSoup

def get_text(post_id):
    # Construct the post URL
    url = f"https://stackoverflow.com/questions/{post_id}"

    # Make the HTTP request and get the HTML content
    response = requests.get(url)
    html_content = response.content

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the title element
    title_element = soup.find('h1', {'class': 'fs-headline1'})
    title = title_element.get_text().strip()

    # Find the post body element
    post_body_elements = soup.find_all('div', {'class': 'js-post-body'})

    # Extract the post body text for each answer element and preprocess
    post_bodies = []
    for post_body_element in post_body_elements:
#         # Remove "s-code-block" class
#         for s_code_block in post_body_element.select('pre'):
#             s_code_block.extract()

#         # Remove HTML tags
#         for tag in post_body_element.find_all():
#             tag.unwrap()

        # Extract the cleaned up post body text
        body_element = post_body_element.get_text()

        post_bodies.append(body_element)

    post_body = ''.join(post_bodies)
    text = title + "\n" + re.sub(r'\n{2,}', ' ', post_body)
    return text

In [2]:
import os
import pandas as pd
import json
import time

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/home/azmain/experiment.csv')

output_folder = '/home/azmain/api_extractor_all_json'

# Initialize a count variable
count = 0

# Loop through each row in the DataFrame and create a JSON file for it
for index, row in df.iterrows():
    time.sleep(10)
    # Extract the values from the relevant columns
    post_id = int(row['Post ID'])
    api = row['API']
    expected_output = row['Expected Output']
    
    post_text = str(get_text(post_id))
    
    # Create a dictionary to represent the JSON object
    data = {
        'post_id': post_id,
        'api': api,
        'expected_output': expected_output,
        'text': post_text
    }
    
    # Increment the count variable
    count += 1
    
    # Construct the file path for the JSON file
    file_path = os.path.join(output_folder, f'{count}_{post_id}.json')
    
    # Write the dictionary to a JSON file
    with open(file_path, 'w') as outfile:
        json.dump(data, outfile)